# Building Machine Learning Classifiers: Evaluate Gradient Boosting with GridSearchCV

**Grid-search:** Exhaustively search all parameter combinations in a given grid to determine the best model.

**Cross-validation:** Divide a dataset into k subsets and repeat the holdout method k times where a different subset is used as the holdout set in each iteration.

### Read in text

In [2]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_tfidf_feat.columns = X_tfidf_feat.columns.astype(str)


# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)
X_count_feat.columns = X_count_feat.columns.astype(str)


X_count_feat.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,49,4.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,62,3.2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,28,7.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,135,4.4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Exploring parameter settings using GridSearchCV

In [3]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [4]:
gb=GradientBoostingClassifier()
param= {
    'n_estimators':[100,150],
    'max_depth':[7,11,15],
    'learning_rate':[0.1]
}

gs = GridSearchCV(gb, param, cv=5, n_jobs=-1) #train models on diffrent subsets and parameter settings in parallel. not the models themselves.
cv_fit= gs.fit(X_tfidf_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,225.151975,11.030691,0.166882,0.044759,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964093,0.979354,0.970350,0.970350,0.966757,0.970181,0.005157,1
5,237.170358,1.783681,0.112943,0.008864,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964991,0.976661,0.970350,0.968553,0.968553,0.969822,0.003837,2
1,172.431716,2.185939,0.333934,0.172181,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.964093,0.978456,0.971249,0.964960,0.967655,0.969283,0.005221,3
4,177.902870,5.102876,0.130607,0.021033,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.965889,0.972172,0.970350,0.968553,0.964960,0.968385,0.002691,4
0,115.641997,2.004190,0.191129,0.017955,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.963196,0.977558,0.970350,0.964960,0.964960,0.968205,0.005258,5


In [5]:
gb=GradientBoostingClassifier()
param= {
    'n_estimators':[100,150],
    'max_depth':[7,11,15],
    'learning_rate':[0.1]
}

gs = GridSearchCV(gb, param, cv=5, n_jobs=-1) #train models on diffrent subsets and parameter settings in parallel. not the models themselves.
cv_fit= gs.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,210.799632,4.544852,0.150684,0.016241,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964991,0.976661,0.968553,0.964061,0.973046,0.969462,0.004789,1
4,170.743663,3.575592,0.127404,0.010224,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.962298,0.978456,0.968553,0.963163,0.971249,0.968744,0.005888,2
1,153.847517,1.417510,0.208783,0.027356,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.963196,0.978456,0.970350,0.964960,0.965858,0.968564,0.005480,3
5,229.725147,0.773008,0.119533,0.009873,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.963196,0.974865,0.968553,0.964960,0.970350,0.968385,0.004112,4
0,103.426002,0.630827,0.211733,0.024575,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.963196,0.979354,0.967655,0.963163,0.968553,0.968384,0.005917,5
